<a href="https://colab.research.google.com/github/Alex-2911/Basketball-Projection/blob/main/_2_01122024_get_data_next_game_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
current_season = 2025


In [14]:
# import os
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define base directory in Google Drive
#DATA_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "data")
#SRC_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "Whole_Statistic")

# Define directories
base_dir = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data"
STANDINGS_DIR = os.path.join(base_dir, f"{current_season}_standings")
SCORES_DIR = os.path.join(base_dir, f"{current_season}_scores")
SRC_DIR = os.path.join(base_dir, f"Whole_Statistic")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from datetime import datetime, timedelta
today = datetime.now() - timedelta(days=0)
#today = datetime.now() - timedelta(days=1)

today_str_format = today.strftime("%Y-%m-%d")


In [16]:
from bs4 import BeautifulSoup
from datetime import datetime
import os
import requests
import calendar
import pandas as pd

# Define the current season and directories
#DATA_DIR = os.path.join("D:\\", "1. Python", "1. NBA Script","2025", "Gathering_Data", "data")
#STANDINGS_DIR = os.path.join(DATA_DIR, f"{current_season}_standings")

# Calculate today's date
now = datetime.now()
today_str = now.strftime("%a, %b ") + str(int(now.strftime("%d"))) + now.strftime(", %Y")
today_date = datetime.strptime(today_str, "%a, %b %d, %Y")

print(f"Today's date: {today_str}")

month_for_coming_games = now.month
month_name_for_coming_games = calendar.month_name[month_for_coming_games].lower()
print(month_name_for_coming_games)

file_name = f"NBA_{current_season}_games-{month_name_for_coming_games}.html"
file_path = os.path.join(STANDINGS_DIR, file_name)
print(f"Checking file: {file_path}")

def scrape_season_for_month(season, month, month_name, standings_dir):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    print(url)

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all("a", href=True)
        month_link = None
        for link in links:
            if f"NBA_{season}_games-{month_name}" in link['href']:
                month_link = "https://www.basketball-reference.com" + link['href']
                break
        if month_link:
            month_response = requests.get(month_link)
            month_response.raise_for_status()
            with open(os.path.join(standings_dir, f"NBA_{season}_games-{month_name}.html"), "w", encoding='utf-8') as f:
                f.write(month_response.text)
            print(f"Data for {month_name.title()} saved.")
        else:
            print(f"No link found for {month_name.title()} {season}.")
    except requests.RequestException as e:
        print(f"An error occurred while fetching data: {e}")

Today's date: Sun, Dec 1, 2024
december
Checking file: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings/NBA_2025_games-december.html


In [17]:
from datetime import datetime
from bs4 import BeautifulSoup
import os

def find_games_for_next_day(today_date, file_paths):
    next_game_info = []
    next_game_date = None  # Placeholder to track the date of the first game found

    for file_path in file_paths:
        if not os.path.exists(file_path):
            continue

        try:
            with open(file_path, "r", encoding='utf-8') as file:
                soup = BeautifulSoup(file.read(), 'html.parser')

            table = soup.find("table", {"id": "schedule"})

            if table:
                rows = table.find_all("tr")
                for row in rows[1:]:  # Skip the header row
                    game_date_tag = row.find("th", {"data-stat": "date_game"})
                    if game_date_tag:
                        game_date_str = game_date_tag.text.strip()
                        try:
                            game_date = datetime.strptime(game_date_str, "%a, %b %d, %Y")
                        except ValueError:
                            continue

                        # If it's the first game on or after today_date, set next_game_date
                        if game_date >= today_date and next_game_date is None:
                            next_game_date = game_date

                        # Collect games for the determined next game date
                        if game_date == next_game_date:
                            cols = row.find_all("td")
                            if len(cols) >= 4:
                                visitor_team = cols[1].text.strip()
                                home_team = cols[3].text.strip()
                                next_game_info.append({
                                    'date': game_date,
                                    'home_team': home_team,
                                    'visitor_team': visitor_team
                                })
                        # Stop once we've collected all games for the next available game day
                        elif next_game_date is not None and game_date > next_game_date:
                            return next_game_info

        except Exception as e:
            print(f"An error occurred while processing the file: {e}")

    return next_game_info

In [18]:
print(file_path)

# Scrape data for the current month if not present
if not os.path.exists(file_path):
    print(f"File does not exist: {file_path}")
    scrape_season_for_month(current_season, month_for_coming_games, month_name_for_coming_games, STANDINGS_DIR)

next_game_info = find_games_for_next_day(today_date, [file_path])

/content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings/NBA_2025_games-december.html


In [19]:
# Prepare to look ahead one month at a time
next_month = month_for_coming_games % 12 + 1  # Handle month transition properly
months_checked = 0
max_months_to_check = 12  # Check up to 12 months ahead
#next_game_info = None

while not next_game_info and months_checked < max_months_to_check:
    month_name_for_coming_games = calendar.month_name[next_month].lower()
    file_name = f"NBA_{current_season}_games-{month_name_for_coming_games}.html"
    file_path = os.path.join(STANDINGS_DIR, file_name)
    print(file_path,'AAAA')

    if not os.path.exists(file_path):
        scrape_season_for_month(current_season, next_month, month_name_for_coming_games, STANDINGS_DIR)

    next_game_info = find_next_game(today_date, [file_path])
    next_month = (next_month % 12) + 1  # Move to the next month
    months_checked += 1

# Display the next game
# if next_game_info:
#     print(f"The next game is on {next_game_info['date'].strftime('%a, %b %d, %Y')}.")
#     print(f"Teams: {next_game_info['visitor_team']} vs {next_game_info['home_team']}")
# else:
#     print("No upcoming games found.")

# Map team names to their codes and create a DataFrame
team_codes = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHO',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

In [20]:
games = []

# Display all the games for today
if next_game_info:
    for game in next_game_info:  # Loop through the list of games
        home_team_name = team_codes.get(game['home_team'], game['home_team'])  # Use 'game' instead of 'next_game_info'
        visitor_team_name = team_codes.get(game['visitor_team'], game['visitor_team'])
        games.append((home_team_name, visitor_team_name, game['date'].strftime("%Y-%m-%d")))

        print(f"The game is on {game['date'].strftime('%a, %b %d, %Y')}.")
        print(f"Teams: {game['visitor_team']} vs {game['home_team']}")
else:
    print("No games found for today.")

# Create DataFrame
games_df = pd.DataFrame(games, columns=['home_team', 'away_team', 'game_date'])

print(games_df)


The game is on Sun, Dec 01, 2024.
Teams: Orlando Magic vs Brooklyn Nets
The game is on Sun, Dec 01, 2024.
Teams: Indiana Pacers vs Memphis Grizzlies
The game is on Sun, Dec 01, 2024.
Teams: Boston Celtics vs Cleveland Cavaliers
The game is on Sun, Dec 01, 2024.
Teams: New Orleans Pelicans vs New York Knicks
The game is on Sun, Dec 01, 2024.
Teams: Miami Heat vs Toronto Raptors
The game is on Sun, Dec 01, 2024.
Teams: Oklahoma City Thunder vs Houston Rockets
The game is on Sun, Dec 01, 2024.
Teams: Los Angeles Lakers vs Utah Jazz
The game is on Sun, Dec 01, 2024.
Teams: Dallas Mavericks vs Portland Trail Blazers
The game is on Sun, Dec 01, 2024.
Teams: San Antonio Spurs vs Sacramento Kings
The game is on Sun, Dec 01, 2024.
Teams: Denver Nuggets vs Los Angeles Clippers
  home_team away_team   game_date
0       BRK       ORL  2024-12-01
1       MEM       IND  2024-12-01
2       CLE       BOS  2024-12-01
3       NYK       NOP  2024-12-01
4       TOR       MIA  2024-12-01
5       HOU       

In [21]:
# Define the target folder path
#target_folder = r'D:\1. Python\1. NBA Script\2025\Gathering_Data\Next_Game'
base_dir = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data"


# Create the folder if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Save the games_df DataFrame to a CSV file in the target folder
games_df.to_csv(os.path.join(base_dir, 'games_df_' + today_str_format + '.csv'))#, index=False)

print(f"games_df has been saved in the folder: {base_dir}")


games_df has been saved in the folder: /content/drive/My Drive/NBA_Script/2025/Gathering_Data
